In [14]:
#@title Import
import pandas as pd
import numpy as np
from functools import partialmethod
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer


import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = (stopwords.words('english'))

pd.DataFrame.head = partialmethod(pd.DataFrame.head, n=2)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
from IPython.display import clear_output
import json

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [58]:
#@title Gather Data
treks = pd.read_csv("Main_data.csv").dropna().drop(columns="Unnamed: 0")
trek_lookup = treks.reset_index()[["index", "trek_name"]]
trek_disc = treks[["trek_name", "description"]]

In [59]:
#@title List Transformer For NLP
def transformer(A):
  return [ps.stem(word.lower().replace(" ", "")) for word in A if word.lower().replace(" ", "") not in stop_words]


In [ ]:
#@title trnasform
trek_disc["description"] = trek_disc["description"].str.replace("\n","").str.split(" ")
trek_disc["description"] = trek_disc["description"].apply(lambda x:transformer(x))
trek_disc["description"] = trek_disc["description"].apply(lambda x:' '.join(x)).str.replace("brief descript", "")

In [31]:
cv=CountVectorizer(max_features=100, stop_words='english')
vectors = cv.fit_transform(trek_disc['description']).toarray()


In [52]:
tfidf_vectorizer = TfidfVectorizer()


In [35]:
similarity = cosine_similarity(vectors)

In [36]:
similarity_df = pd.DataFrame(similarity)


In [62]:
similarity_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
0,1.0,0.278471,0.230381,0.18315,0.274729,0.19986,0.189987,0.346798,0.391228,0.416228,0.246308,0.239739,0.260963,0.231015,0.304031,0.335881,0.254282,0.294431,0.316814,0.225199,0.219627,0.102101,0.323505,0.225695,0.335344,0.194613,0.177609,0.150842,0.22973,0.225182,0.215203,0.237962,0.191932,0.255517,0.258438,0.294175,0.320185,0.205303,0.290509,0.368834,0.24503,0.323351,0.344819,0.27075,0.283244,0.095551,0.371596,0.342776,0.284053,0.118608,0.124556,0.12691,0.178307,0.342027,0.306665,0.370456,0.274069,0.258654,0.269432,0.279399,0.228283,0.364528,0.25205,0.435316,0.329534,0.22598,0.320705,0.288912,0.325781,0.352092,0.30676,0.197516,0.454544


In [49]:
def reccomend_trek(trek_name):
  index = trek_lookup[trek_lookup['trek_name']==trek_name].index.values[0]
  distances = pd.DataFrame(similarity_df.loc[index])
  suggestions_index = distances.sort_values(by=index,ascending=False).index[:3]
  suggestions_name = trek_lookup[trek_lookup.index.isin(suggestions_index)]['trek_name'].values
  return(suggestions_name)

In [50]:
reccomend_trek("Friendship Peak Expedition")

array(['Friendship Peak Expedition', 'Black Peak (Kalanag) Expedition',
       'Nag Tibba Trek'], dtype=object)

In [64]:
trek_lookup.to_pickle("TreksList.pkl")
similarity_df.to_pickle("similarity_df.pkl")